In [1]:
import pickle
import numpy as np
import random

In [2]:
ratings = pickle.load(open('items_10000_matrix.pickle', 'rb'), encoding='iso-8859-1')

In [3]:
type(ratings)

scipy.sparse.csr.csr_matrix

In [4]:
ratings = ratings.toarray()

In [5]:
ratings.shape

(71869, 10000)

*ratings* -  np.array состоящий из нулей и единиц, где каждая строка соответсвует пользователю, а каждая колонка - книге.  
1 - на пересечении строки i колонки j значит, что пользователь i прочитал книгу j. 0 - пользователь i не читал книгу j.

# Part 1

### Task 1. Найти самую популярную книгу.

In [8]:
book_readers_count = ratings.sum(axis=0)
most_popular_book = book_readers_count.argmax()
print('most popular book = {} (numeration from 0)'.format(most_popular_book))

most popular book = 20 (numeration from 0)


### Task 2. Найти пользователя со второй по величине библиотекой.

In [9]:
readers_library = ratings.sum(axis=1)
second_biggest_library = readers_library.argsort()[-2]
print('user_id with the second biggest library = {}'.format(second_biggest_library))

user_id with the second biggest library = 69648


### Task 3. Найти среднее и медиану размера библиотеки.

In [10]:
mean = readers_library.mean()
print('mean = {}'.format(mean))

median = np.median(readers_library)
print('median = {}'.format(median))

mean = 2.266693567463023
median = 2.0


### Task 4. Найти медиану размера библиотеки у пользователей, которые прочитали книгу номер с id 100.

In [11]:
readers_100_library = list((x.sum() for x in ratings if x[100] == 1))
median_number = np.median(readers_100_library)
print('Median number of books of users who read the book with id 100 = {}'.format(median_number))

Median number of books of users who read the book with id 100 = 3.0


# Part 2

### Task 1. Похожие книги.
1. Найди 20 наиболее похожих книг для книги с id = 100. В качестве меры похожести используй cosine similarity. Используй наиболее быстрый вариант подсчета.
  
$$cosine\ similarity = \frac{A\cdot B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n} A_iB_i}{\sqrt{\sum_{i=1}^{n}A_i^2}\sqrt{\sum_{i=1}^{n}B_i^2}}$$  
  
В качестве ответа верни список кортежей, где каждый кортеж представляет собой пару (book_id, similarity).

In [12]:
# будем использовать говую библиотеку, которая наверняка оптимизирована и проверена временем
# поэтому это более быстрый способ, чем если бы я писала это ручками)

from scipy import spatial
import math

In [13]:
ratings_t = ratings.T

In [14]:
vector_100 = ratings_t[100]
print('Количество пользователей, прочитавших книгу 100:', vector_100.sum())


Количество пользователей, прочитавших книгу 100: 14


Если книга читаемая, то книги, которые не читались явно не будут ей близки. Поскольку пользователей много, то расстояние будет вычисляться долго. Значит, вычислять расстояние будем только тогда, когда книгу прочитал хотя бы 1 человек

In [15]:
similarities = []
# сразу запомним непрочитанные книги
zero_books = [] 
for book in range(10000):
    s = ratings_t[book].sum()
    
    if (s != 0):
        similarity = 1 - spatial.distance.cosine(ratings_t[book], vector_100)
        similarities.append((book, similarity))
    else:
        zero_books.append(book)

In [16]:
books_result = list(filter(lambda x: x[0] != 100, similarities))
books_result.sort(key=lambda x: x[1], reverse=True)
print("Closest books to 100:")
books_result[:20]

Closest books to 100:


[(5819, 0.2672612419124244),
 (5863, 0.2672612419124244),
 (5134, 0.18898223650461365),
 (2370, 0.13363062095621214),
 (3940, 0.13363062095621214),
 (5440, 0.13363062095621214),
 (3468, 0.10910894511799618),
 (5546, 0.08451542547285162),
 (214, 0.06482037235521643),
 (1495, 0.06482037235521643),
 (6, 0.048795003647426616),
 (528, 0.04454354031873742),
 (251, 0.03479445003196102),
 (836, 0.031943828249996975),
 (426, 0.030656966974248245),
 (312, 0.030261376633440085),
 (446, 0.025141822757713483),
 (73, 0.02126216277812809),
 (191, 0.01667129822620217),
 (76, 0.015667131465128104)]

Посмотрим, нашлись ли непрочтенные книги

In [23]:
len(zero_books)

4071

Довольно много непрочитанных книг. Значит, мы очень хорошо улучшили скорость подсчета вектора близостей для книг)))

# Part 3

### Task 1. Матрица похожих книг (топ-20).

Создать матрицу похожих книг (матрица 10000 x 10000) использую метрику *cosine similarity*. В каждой строке оставить только топ 20 самых похожих. Лучше написать функцию которая на вход получает матрицу рейтингов на выходе матрица похожих книг. Эту фунцкцию можно будет использовать в следующем задании.

Если книгу никто не читал, то бесполезно вычислять косинуное расстояние, поскольку оно везде будет nan (деление на 0, если все значения 0). Поэтому всем книгам, которые не были прочитаны ближайшими будем ставить непрочитанные книги (например, это новинки)

In [36]:
# функция подсчета близости для одной книги
def compute_book_similarity(id_book, rating, zero_books):
    similarities = []
    current_book = ratings_t[id_book]    
    
    
    for book in range(10000):
        if book not in zero_books:
            similarity = 1 - spatial.distance.cosine(rating[book], current_book)
            similarities.append((book, similarity))
    
    books_result = list(filter(lambda x: x[0] != id_book, similarities))
    books_result.sort(key=lambda x: x[1], reverse=True)
            
    return books_result[:20]  
    
# функция определения матрицы близости
def compute_sim_matrix(ratings, zero_books):  
    r_t = ratings.T
    matrix = []
        
    for book in range(10000):
        similar_books = [0] * 10000
        
        if book not in zero_books:
            similar = compute_book_similarity(book, r_t, zero_books)
            
            for i in range(20):
                similar_books[similar[i][0]] = similar[i][1]
        else:
            # для непрочитанных книг определим 20 других непрочитанных книг
            zero_books.remove(book)
            other_books = random.choices(zero_books, k=20)
            
            # будем считать книги очень похожими, если их никто не прочитал            
            for i in range (20):
                similar_books[other_books[i]] = 1
                
            zero_books.append(book)
            
        matrix.append(similar_books)
            

In [ ]:
compute_sim_matrix(ratings, zero_books)

### Task 2. Оценить рекомендательную систему.

Оценить качество рекомендательной системы, основанной на матрице похожих книг, использую метрики **precision** и **recall**. Рекомендации должны получаться путем умножения матрицы похожих книг на вектор пользователя.
  
**precison** - кол-во книг которые попали пользователю в рекомендации и он их действительно добавил делить на кол-во рекомендованных книг.
  
**recall** - кол-во книг которые попали пользователю в рекомендации и он их действительно добавил делить на кол-во книг добавленных пользователем. 
  
Метрики считать на 20 самых лучших рекомендациях для каждого пользователя.